In [1]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [3]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [4]:
async def get_expiry(f_today, index):

    if index == 'bnf':    
        if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
            f_expiry = dt.date(2024, 1, 25)
        elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
            f_expiry = dt.date(2024, 1, 31)
        elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
            f_expiry = dt.date(2024, 2, 29)
        elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
            f_expiry = dt.date(2024, 2, 27)
        elif f_today < dt.date(2023, 9, 1):
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
        elif f_today >= dt.date(2023, 9, 1):
            if f_today.day < 24:
                days_to_wednesday = (2 - f_today.weekday()) % 7
                nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
                f_expiry = nearest_wednesday
                if nse.valid_days(
                    start_date=nearest_wednesday, end_date=nearest_wednesday
                ).empty:
                    f_expiry = nearest_wednesday - dt.timedelta(days=1)
            else:
                days_to_thursday = (3 - f_today.weekday()) % 7
                nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
                f_expiry = nearest_thursday
                if nse.valid_days(
                    start_date=nearest_thursday, end_date=nearest_thursday
                ).empty:
                    f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'nifty':
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'finnifty' or index == 'fnf':
        days_to_thursday = (1 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'midcpnifty' or index == 'midcp':
        days_to_thursday = (0 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

async def get_expiry_nifty(f_today):

    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"


def get_option_contract_name2(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [5]:
bnf_1min = pd.read_csv("../data/nifty_30min_tv.csv")
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["time"])
bnf_1min = bnf_1min[bnf_1min["datetime"].dt.year >= 2017]

In [6]:
bnf_1min.tail()

,time,open,high,low,close,datetime
22336,2024-12-18T13:15:00+05:30,24221.5000,24235.6500,24176.1000,24178.3500,2024-12-18 13:15:00+05:30
22337,2024-12-18T13:45:00+05:30,24179.4500,24222.4000,24168.0500,24196.6500,2024-12-18 13:45:00+05:30
22338,2024-12-18T14:15:00+05:30,24196.3500,24210.8000,24166.7000,24183.4500,2024-12-18 14:15:00+05:30
22339,2024-12-18T14:45:00+05:30,24183.5500,24204.4000,24172.1000,24189.3000,2024-12-18 14:45:00+05:30
22340,2024-12-18T15:15:00+05:30,24189.0000,24215.3000,24188.3500,24214.7000,2024-12-18 15:15:00+05:30


In [7]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [8]:
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"])
list_of_traded_dates = set(bnf_1min["datetime"].dt.date)
list_of_traded_dates

{datetime.date(2021, 3, 18),
 datetime.date(2020, 3, 6),
 datetime.date(2021, 10, 28),
 datetime.date(2020, 3, 26),
 datetime.date(2021, 12, 7),
 datetime.date(2023, 8, 24),
 datetime.date(2024, 7, 1),
 datetime.date(2024, 11, 25),
 datetime.date(2018, 8, 10),
 datetime.date(2024, 11, 28),
 datetime.date(2020, 9, 9),
 datetime.date(2024, 7, 8),
 datetime.date(2018, 9, 21),
 datetime.date(2022, 12, 12),
 datetime.date(2018, 5, 24),
 datetime.date(2024, 8, 6),
 datetime.date(2018, 5, 23),
 datetime.date(2022, 6, 13),
 datetime.date(2018, 7, 11),
 datetime.date(2022, 4, 6),
 datetime.date(2020, 9, 21),
 datetime.date(2021, 4, 13),
 datetime.date(2020, 1, 24),
 datetime.date(2022, 4, 25),
 datetime.date(2022, 6, 22),
 datetime.date(2022, 8, 1),
 datetime.date(2024, 10, 9),
 datetime.date(2020, 3, 23),
 datetime.date(2019, 3, 13),
 datetime.date(2022, 7, 4),
 datetime.date(2018, 2, 2),
 datetime.date(2023, 6, 21),
 datetime.date(2018, 3, 15),
 datetime.date(2019, 10, 17),
 datetime.date(202

In [20]:
import pandas as pd

def calculate_signals(df, n, rsi_period, rsi_overbought, ema_length):
    # Calculate RSI
    def calculate_rsi(series, period):
        # Calculate the price changes
        delta = series.diff()
    
        # Separate gains and losses
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
    
        # Calculate the first average gain and loss (Simple Moving Average for the first period)
        avg_gain = gain.rolling(window=period, min_periods=period).mean()
        avg_loss = loss.rolling(window=period, min_periods=period).mean()
    
        # Calculate the subsequent smoothed averages (similar to EMA)
        for i in range(period, len(series)):
            avg_gain.iloc[i] = (avg_gain.iloc[i - 1] * (period - 1) + gain.iloc[i]) / period
            avg_loss.iloc[i] = (avg_loss.iloc[i - 1] * (period - 1) + loss.iloc[i]) / period
    
        # Calculate the Relative Strength (RS) and the RSI
        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))
    
        return rsi

    # Apply the TradingView-style RSI calculation
    df['RSI'] = calculate_rsi(df['close'], rsi_period)

    # Calculate the previous n candles' low
    df['Prev_N_Low'] = df['low'].rolling(window=n).min().shift(1)
    df['ema'] = df['close'].ewm(span=ema_length, adjust=False).mean()

    # Signal generation logic
    df['Sell_Signal'] = (
        (df['close'] < df['Prev_N_Low']) &
        (df['RSI'] > rsi_overbought)
    )

    return df

In [21]:
bnf_1min.head()

,time,open,high,low,close,datetime,RSI,Prev_N_Low,ema,Sell_Signal
0,2018-01-01T09:15:00+05:30,10533.7500,10535.5000,10516.3500,10523.8500,2018-01-01 09:15:00+05:30,NaN,NaN,10523.8500,False
1,2018-01-01T09:45:00+05:30,10524.0500,10533.0500,10521.8500,10532.2500,2018-01-01 09:45:00+05:30,NaN,NaN,10525.3773,False
2,2018-01-01T10:15:00+05:30,10532.0500,10537.8500,10527.5500,10531.8500,2018-01-01 10:15:00+05:30,NaN,10516.3500,10526.5541,False
3,2018-01-01T10:45:00+05:30,10532.3500,10536.3000,10525.2500,10527.4500,2018-01-01 10:45:00+05:30,NaN,10521.8500,10526.7170,False
4,2018-01-01T11:15:00+05:30,10527.9000,10534.2000,10527.4500,10532.6000,2018-01-01 11:15:00+05:30,NaN,10525.2500,10527.7867,False


In [22]:
lev_ = 8
portfolio_value = 10000000
lot_size_ = 75
slippage_ = 0.0002

In [23]:
def backtest_short(df, n, rsi_period, rsi_overbought, ema_length):
    trade_book = []
    in_trade = False
    points = 0
    is_trailing_active = False
    trailing_sl = 0
    df = calculate_signals(df, n, rsi_period, rsi_overbought, ema_length)
    # print(df[df['Sell_Signal']].to_string())
    for i in range(n, len(df)):
        # print(i, df.loc[i], df.loc[i, 'open'])
        # break
        if df.loc[i, 'Sell_Signal'] and not in_trade:
            # Entry Triggered
            entry_price = df.loc[i, 'close']
            entry_time = df.loc[i, 'datetime']
            initial_sl = df.loc[i-n:i, 'high'].max()
            sl_in_points = initial_sl - entry_price
            target = 5 * sl_in_points
            in_trade = True

        if in_trade:
            if df.loc[i, 'high'] < df.loc[i, 'ema']:
                is_trailing_active = True
            if is_trailing_active:
                trailing_sl = df.loc[i, 'ema']
            # print(df.loc[i, 'open'], initial_sl)
            # break
            if df.loc[i, 'open'] > initial_sl and not points:
                # Gap condition
                if df.loc[i, 'close'] < initial_sl:
                    initial_sl = df.loc[i, 'high']
                else:
                    exit_price = df.loc[i, 'close']
                    exit_time = df.loc[i, 'datetime']
                    in_trade = False
                    remark = 'Gap SL'
                    points = entry_price - exit_price
            if df.loc[i, 'high'] > initial_sl and not points:
                # Initial SL Hit
                exit_price = df.loc[i, 'close']
                exit_time = df.loc[i, 'datetime']
                in_trade = False
                remark = 'Initial SL'
                points = entry_price - exit_price
            if is_trailing_active:
                if df.loc[i, 'close'] > trailing_sl and not points:
                    # TSL Hit
                    exit_price = df.loc[i, 'close']
                    exit_time = df.loc[i, 'datetime']
                    in_trade = False
                    remark = 'Trailing SL'
                    points = entry_price - exit_price
            if df.loc[i, 'low'] < entry_price - target and not points:
                # Target Hit
                exit_price = entry_price - target
                exit_time = df.loc[i, 'datetime']
                in_trade = False
                remark = 'Target'
                points = entry_price - exit_price

        if points:
            index_lev = lev_
            qty = int(round(portfolio_value * index_lev / entry_price / lot_size_)) * lot_size_
            slippage = slippage_ * (entry_price + exit_price)
            final_points = points - slippage
            trade = {
                "Trade Type": "Short",
                "Entry Time": entry_time,
                "Entry Price": entry_price,
                "Initial SL": initial_sl,
                "Trailing SL": trailing_sl,
                "Exit Time": exit_time,
                "Exit Price": exit_price,
                "Points Captured": points,
                "Slippage in Points": slippage,
                "After Costs": final_points,
                "PnL": final_points * qty,
                "Remarks": remark,
                "Qty": qty,
                "Leverage": index_lev,
                "ROI%": (final_points * qty / portfolio_value) * 100,
                "Trade Year": entry_time.year,
                "Trade Month": entry_time.month,
            }
            # print(trade)
            trade_book.append(trade)
            points = 0
            in_trade = False
            is_trailing_active = False
            trailing_sl = 0

    return pd.DataFrame(trade_book)
            

In [24]:
tb = backtest_short(bnf_1min, n=3, rsi_period=5, rsi_overbought=50, ema_length=14)

In [25]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2018, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2018, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation
    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio : stats_df8}

In [26]:
variation = 1
stats = generate_stats(tb, variation)
for x, y in stats.items():
    final_stats = y

final_stats

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2018,19.4713,23,52.1739,3.9669,-2.5574,-9.8137,1.9841,1
2019,10.5576,15,40.0000,6.3776,-3.0787,-20.2631,0.5210,1
2020,16.2460,22,45.4545,7.6380,-5.0112,-34.0476,0.4772,1
2021,-7.4359,26,38.4615,4.5754,-3.3244,-15.9534,-0.4661,1
2022,-53.8414,33,36.3636,2.7584,-4.1401,-59.6103,-0.9032,1
2023,-10.6684,18,27.7778,5.8601,-3.0745,-33.8708,-0.3150,1
2024,24.4648,17,52.9412,5.5168,-3.1484,-19.8815,1.2305,1
Overall,-1.2060,154.0000,41.5584,5.0009,-3.5696,-124.2861,-0.0097,1.0000


In [27]:
tb[tb['Trade Year'] > 2023]

,Trade Type,Entry Time,Entry Price,Initial SL,Trailing SL,Exit Time,Exit Price,Points Captured,Slippage in Points,After Costs,PnL,Remarks,Qty,Leverage,ROI%,Trade Year,Trade Month
137,Short,2024-01-04 14:15:00+05:30,21644.7000,21674.3000,0.0000,2024-01-04 15:15:00+05:30,21669.6000,-24.9000,8.6629,-33.5629,-123343.5105,Initial SL,3675,8,-1.2334,2024,1
138,Short,2024-01-05 10:45:00+05:30,21693.1500,21763.9500,21589.2877,2024-01-09 09:15:00+05:30,21607.7000,85.4500,8.6602,76.7898,282202.6253,Trailing SL,3675,8,2.8220,2024,1
139,Short,2024-01-19 10:45:00+05:30,21586.9000,21720.3000,21627.8834,2024-01-23 09:15:00+05:30,21722.3000,-135.4000,8.6618,-144.0618,-529427.2620,Gap SL,3675,8,-5.2943,2024,1
140,Short,2024-03-07 10:15:00+05:30,22443.8000,22523.6500,0.0000,2024-03-07 13:45:00+05:30,22499.2000,-55.4000,8.9886,-64.3886,-231798.9600,Initial SL,3600,8,-2.3180,2024,3
141,Short,2024-03-14 13:15:00+05:30,22115.8500,22204.6000,22000.5363,2024-03-18 11:45:00+05:30,22034.1000,81.7500,8.8300,72.9200,262512.0360,Trailing SL,3600,8,2.6251,2024,3
142,Short,2024-04-09 11:45:00+05:30,22719.9500,22768.4000,22672.0021,2024-04-10 09:15:00+05:30,22720.7500,-0.8000,9.0881,-9.8881,-34855.6935,Trailing SL,3525,8,-0.3486,2024,4
143,Short,2024-04-22 12:45:00+05:30,22273.4500,22312.1000,0.0000,2024-04-22 14:15:00+05:30,22313.5000,-40.0500,8.9174,-48.9674,-176282.6040,Initial SL,3600,8,-1.7628,2024,4
144,Short,2024-04-26 09:45:00+05:30,22538.1500,22615.5000,22477.9306,2024-04-29 09:15:00+05:30,22488.1000,50.0500,9.0053,41.0448,144682.7438,Trailing SL,3525,8,1.4468,2024,4
145,Short,2024-04-30 12:45:00+05:30,22714.7000,22756.1000,0.0000,2024-04-30 13:45:00+05:30,22768.0500,-53.3500,9.0966,-62.4465,-220124.0887,Initial SL,3525,8,-2.2012,2024,4
146,Short,2024-06-06 11:45:00+05:30,22720.7500,22910.1500,0.0000,2024-06-07 09:15:00+05:30,22942.5000,-221.7500,9.1326,-230.8827,-813861.3413,Initial SL,3525,8,-8.1386,2024,6


In [28]:
stats_dictionary = {}
rsi_lengths = [14, 9, 21, 6, 12, 16]
for i in range(2, 5):
    for j in rsi_lengths:
        for k in range(30, 81, 5):
            for l in range(5, 36, 5):
                variation = f'n {i}, rsi {j}, threshold {k}, ema {l}'
                print(variation)
                tb = backtest_short(bnf_1min, i, j, k, l)
                if len(tb) > 0:
                    stats = generate_stats(tb, variation)
                    for x, y in stats.items():
                        if x > 8:
                            final_stats = y
                            print(final_stats.to_string())
                            stats_dictionary[x] = y


n 2, rsi 14, threshold 30, ema 5
n 2, rsi 14, threshold 30, ema 10
n 2, rsi 14, threshold 30, ema 15
n 2, rsi 14, threshold 30, ema 20
n 2, rsi 14, threshold 30, ema 25
n 2, rsi 14, threshold 30, ema 30
n 2, rsi 14, threshold 30, ema 35
n 2, rsi 14, threshold 35, ema 5
n 2, rsi 14, threshold 35, ema 10
n 2, rsi 14, threshold 35, ema 15
n 2, rsi 14, threshold 35, ema 20
n 2, rsi 14, threshold 35, ema 25
n 2, rsi 14, threshold 35, ema 30
n 2, rsi 14, threshold 35, ema 35
n 2, rsi 14, threshold 40, ema 5
n 2, rsi 14, threshold 40, ema 10
n 2, rsi 14, threshold 40, ema 15
n 2, rsi 14, threshold 40, ema 20
n 2, rsi 14, threshold 40, ema 25
n 2, rsi 14, threshold 40, ema 30
n 2, rsi 14, threshold 40, ema 35
n 2, rsi 14, threshold 45, ema 5
n 2, rsi 14, threshold 45, ema 10
n 2, rsi 14, threshold 45, ema 15
n 2, rsi 14, threshold 45, ema 20
n 2, rsi 14, threshold 45, ema 25
n 2, rsi 14, threshold 45, ema 30
n 2, rsi 14, threshold 45, ema 35
n 2, rsi 14, threshold 50, ema 5
n 2, rsi 14, thresh

/tmp/ipykernel_35588/873016624.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


n 2, rsi 14, threshold 70, ema 10


/tmp/ipykernel_35588/873016624.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


n 2, rsi 14, threshold 70, ema 15


...

        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio                          Variation
2018       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  n 4, rsi 16, threshold 75, ema 25
2019       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  n 4, rsi 16, threshold 75, ema 25
2020      29.9326            1 100.0000               29.9326                 NaN       0.0000          inf  n 4, rsi 16, threshold 75, ema 25
2021       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  n 4, rsi 16, threshold 75, ema 25
2022       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  n 4, rsi 16, threshold 75, ema 25
2023       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  n 4, rsi 16, threshold 75, ema 25

/tmp/ipykernel_35588/873016624.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)
/tmp/ipykernel_35588/873016624.py:70: RuntimeWarning: divide by zero encountered in scalar divide
  overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)


        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio                          Variation
2018       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  n 4, rsi 16, threshold 75, ema 30
2019       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  n 4, rsi 16, threshold 75, ema 30
2020      29.9326            1 100.0000               29.9326                 NaN       0.0000          inf  n 4, rsi 16, threshold 75, ema 30
2021       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  n 4, rsi 16, threshold 75, ema 30
2022       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  n 4, rsi 16, threshold 75, ema 30
2023       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  n 4, rsi 16, threshold 75, ema 30

/tmp/ipykernel_35588/873016624.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)
/tmp/ipykernel_35588/873016624.py:70: RuntimeWarning: divide by zero encountered in scalar divide
  overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)


        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio                          Variation
2018       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  n 4, rsi 16, threshold 75, ema 35
2019       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  n 4, rsi 16, threshold 75, ema 35
2020      29.9326            1 100.0000               29.9326                 NaN       0.0000          inf  n 4, rsi 16, threshold 75, ema 35
2021       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  n 4, rsi 16, threshold 75, ema 35
2022       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  n 4, rsi 16, threshold 75, ema 35
2023       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  n 4, rsi 16, threshold 75, ema 35

In [18]:
stats_dictionary

{}